In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

K = 3
STITCH_BATCH_SIZE = 32 # todo study the effect
MAX_DEPTH = 16
THRESOULD = 0.8
TOTAL_THRESOULD = 0.8
EVAL_BATCH_SIZE = 16
EXP_TIME = int(time.time())

In [15]:
# dataset_train
# dataset_val

In [25]:
from datasets import load_from_disk
from stitchnet.stitchonnx.utils import run_experiment, get_stitching_data_from_dataset

i = 0
for i in range(5):
    dataset_train = load_from_disk(f'./_data/food101_set{i}/train', keep_in_memory=True)
    dataset_val = load_from_disk(f'./_data/food101_set{i}/validation', keep_in_memory=True)

    RESULT_NAME = f"{EXP_TIME}_result_food101/set{i}_CKA_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}_K_{K}"

    data_score = get_stitching_data_from_dataset(dataset_train, batch_size=STITCH_BATCH_SIZE)

    run_experiment(result_name=RESULT_NAME, 
                   nets=nets, 
                   data_score=data_score, 
                   dataset_val=dataset_val, 
                   dataset_train=dataset_train, 
                   scoring_method='CKA', 
                   branching_factor=K,
                   threshold=THRESOULD, 
                   total_threshold=TOTAL_THRESOULD, 
                   max_depth=MAX_DEPTH, 
                   eval_batch_size=EVAL_BATCH_SIZE)

100%|██████████| 32/32 [00:02<00:00, 13.71it/s]


current depth: 1
potential next fragments: 3
potential next fragments before thresholding of 0.8: 3 ['1.0', '0.6', '0.59']
potential next fragments after filter duplicated fragments: 3 ['1.0', '0.6', '0.59']
potential next fragments after thresholding of 0.8: 1 ['1.0']
totalscore 1.0
diff sampled tensor(0.)
epoch 0 loss 6.969868395283225e-11 torch.Size([23328, 64]) torch.Size([23328, 64])
(32, 3, 224, 224)
current depth: 2
potential next fragments: 3
potential next fragments before thresholding of 0.8: 3 ['1.0', '0.59', '0.57']
potential next fragments after filter duplicated fragments: 3 ['1.0', '0.59', '0.57']
potential next fragments after thresholding of 0.8: 1 ['1.0']
totalscore 1.0
diff sampled tensor(0.0019)
epoch 0 loss 1.5265403865900623e-07 torch.Size([5408, 192]) torch.Size([5408, 192])
(32, 3, 224, 224)
current depth: 3
potential next fragments: 3
potential next fragments before thresholding of 0.8: 3 ['1.0', '0.77', '0.74']
potential next fragments after filter duplicated 

100%|██████████| 10/10 [00:00<00:00, 74.03it/s]


(319, 1000) (319,)


100%|██████████| 1/1 [00:00<00:00,  2.16it/s]

...

(299, 1000) (299,)


 50%|█████     | 1/2 [00:00<00:00,  1.57it/s]

(32, 3, 224, 224)
(5, 3, 224, 224)


100%|██████████| 2/2 [00:00<00:00,  2.46it/s]


accuracy 0.6486486486486487
saving to _results/1689277241_result_food101/set4_CKA_BS_32_MD_16_T_0.8_TT_0.8_K_3/net022


In [28]:
print('DONE RUNNING')

DONE RUNNING


In [41]:
from evaluate import evaluator
from datasets import load_from_disk
from stitchnet.stitchonnx.report import NetEvalPipeline
from stitchnet.stitchonnx.utils import evaluate_net, load_dl, get_stitching_data_from_dataset, get_macs_params_onnx, get_score_net
from stitchnet.stitchonnx.viz import draw_stitchNet_fromTuples

# evaluate original networks
for setI in range(5):
    dataset_train = load_from_disk(f'./_data/food101_set{setI}/train', keep_in_memory=True)
    dataset_val = load_from_disk(f'./_data/food101_set{setI}/validation', keep_in_memory=True)

    data_score = get_stitching_data_from_dataset(dataset_train, batch_size=STITCH_BATCH_SIZE)

    RESULT_NAME = f"_results/{EXP_TIME}_result_food101/set{setI}_original"
    os.makedirs(RESULT_NAME, exist_ok=True)

    modelnames = sorted(glob('_models/*.onnx'))
    for i,modelname in tqdm(enumerate(modelnames)):
        name = os.path.basename(modelname)
        namewithoutext = os.path.splitext(name)[0]

        model_onnx1 = load_onnx_model(modelname)
        macs, params = get_macs_params_onnx(model_onnx1)

        # get CKA score of the original network, no stitching
        fragmentFiles = sorted(glob(f'_models/fragments/net{i:03}/*.onnx'))
        onnxFragments = []
        js = []
        for j,fragmentFile in enumerate(fragmentFiles):
            onnxFragment = load_onnx_model(fragmentFile)
            onnxFragments.append(onnxFragment)
            js.append((i,j))
        net1 = Net(onnxFragments, i)
        score = get_score_net(net1, data_score)

        net = Net([model_onnx1])
        label_column = 'label'

        # evaluate the original network
        valacc, trainacc = evaluate_net(net, dataset_train, dataset_val, label_column=label_column)

        print('VALACC:', valacc, 'TRAINACC:', trainacc)

        with open(f'{RESULT_NAME}/{namewithoutext}.txt', 'w') as f:
            f.write(f'{valacc},{trainacc},{macs},{params},{score},"{tuple(js)}"\n')

        draw_stitchNet_fromTuples(js, name=f'{RESULT_NAME}/{namewithoutext}')

100%|██████████| 32/32 [00:01<00:00, 16.69it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

(32, 3, 224, 224)
(32, 64, 27, 27)
(32, 192, 13, 13)
(32, 384, 13, 13)
(32, 256, 13, 13)
(32, 9216)
(32, 4096)
(32, 4096)



1it [00:01,  1.59s/it]
2it [00:01,  1.40it/s]

input.4 input.4 1.0
input.12 input.12 0.9999999403953552



3it [00:01,  2.24it/s]
4it [00:01,  3.12it/s]

...

2023-07-14 03:43:53.160764878 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {11,1000} for output 70

100%|██████████| 10/10 [00:17<00:00,  1.76s/it]
5it [03:04, 41.73s/it]

(11, 3, 224, 224)
VALACC: 0.7567567567567568 TRAINACC: 0.8060200668896321


/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_99 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_101 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_102 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.co

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1462643/1042804037.py", line 40, in <cell line: 8>
    valacc, trainacc = evaluate_net(net, dataset_train, dataset_val, label_column=label_column)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1652, in evaluate_net
    pipe = NetEvalPipeline(dataset_train, net, label=label_column)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/report.py", line 80, in __init__
    self.ort_sess1 = ort.InferenceSession(fragmentC.fragment.SerializeToString(), providers=['CUDAExecutionProvider'])
  File "/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 347, in __init__
    self._create_inference_session(providers, provider_optio

In [38]:
# RESULT_NAME

In [32]:
# # range(1)
# def runExperiment():
#     k = 0
#     if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
#         with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
#             k = len(f.read().split('\n'))
#     from tqdm import tqdm
#     from stitchnet.stitchonnx.report import Report, ReportPlants, ReportKNNHFDataset
#     import traceback
#     scoreMapper = ScoreMapper(nets, data_score, scoring_method='CKA')
#     with ReportKNNHFDataset(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
#         # for _ in tqdm(range(50)):
#         generator = generate_networks(nets, scoreMapper, data_score, 
#                               threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
#                               maxDepth=MAX_DEPTH, sample=False, K=K)
#         for i,(s,net) in enumerate(generator):
#             try:
#                 netname = f"_results/{RESULT_NAME}/net{k:03}"
#                 report.evaluate(nets, net, netname, s, dataset_val, dataset_train)
#                 net.save(netname)
#                 k += 1
#             except Exception as e:
#                 print('ERROR', e)
#                 traceback.print_exc()
#                 pass

In [68]:
# from torch.utils.data import TensorDataset



In [69]:
# for x in dataset_train:
#     print(x)
#     break

In [70]:
# for x in dl:
#     print(x[0].squeeze(1).shape, x[1].shape)
#     break

In [71]:
# len(dataset_train)

In [72]:
# report.evaluate(nets, net, netname, s, dataset_val, dataset_train)

In [73]:
# import onnxruntime as ort
# ort.get_available_providers()

In [74]:
# for x in dataset_train:
#     print(x)
#     break

In [75]:
# from stitchnet.stitchonnx.utils import accuracy_score_net_plants

In [76]:
# torch.tensor([1,2]).numpy().tobytes()

In [77]:
# for x,t in dataset_val:
#     print(t)

In [78]:
# dataset_val[0]

In [79]:
# accuracy_score_net_plants(net, dataset_val, dataset_train)

In [80]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(np.ones([100,10]), np.ones([100,1]))


In [81]:
# y = knn.predict(np.ones([1,10]))
# y

In [84]:
# dataset_val

In [83]:
print('DONE')

DONE
